In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# The file datasets/attack_attempts.csv should have the format:
# pokemon_name,attack_type,result

data = np.genfromtxt('datasets/attack_attempts.csv', delimiter=',', dtype=str, skip_header=1)

# Split columns
pokemon_names = data[:, 0]
attack_types = data[:, 1]
results = data[:, 2]

## Codificar os dados (Label Encoding)

In [ ]:
pokemon_encoder = LabelEncoder()
attack_encoder = LabelEncoder()
result_encoder = LabelEncoder()

pokemon_encoded = pokemon_encoder.fit_transform(pokemon_names)
attack_encoded = attack_encoder.fit_transform(attack_types)
result_encoded = result_encoder.fit_transform(results)

# Combine Pokémon and Attack Type as input
X = np.column_stack((pokemon_encoded, attack_encoded))
y = result_encoded

## Dividir em treino e teste

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Train the Random Forest model

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Avalueate the model on the test set
accuracy = model.score(X_test, y_test)
print(f"Test accuracy: {accuracy:.2f}")

## Save the trained model and encoders to disk

In [ ]:
import pickle

# Create a dictionary to save everything needed for production
export_dict = {
    "model": model,
    "pokemon_encoder": pokemon_encoder,
    "attack_encoder": attack_encoder,
    "result_encoder": result_encoder
}

# Save it
with open('models/attack_predictor.pkl', 'wb') as f:
    pickle.dump(export_dict, f)

print("Model exported successfully to models/attack_predictor.pkl")

## Example of using the trained model

In [ ]:
# Load the saved model
with open('models/attack_predictor.pkl', 'rb') as f:
    saved_data = pickle.load(f)

# Retrieve the saved model and encoders
model = saved_data["model"]
pokemon_encoder = saved_data["pokemon_encoder"]
attack_encoder = saved_data["attack_encoder"]
result_encoder = saved_data["result_encoder"]

: 

In [ ]:
# Detected Pokemons and Attack Types
detected_pokemon_name = "Pidgey"
available_attack_types = ["Electric", "Water", "Fire", "Grass"]

# For each available attack, predict the result
best_attack = None
best_score = None

for attack in available_attack_types:
    try:
        # Encode the Pokemon name and attack type
        pokemon_encoded = pokemon_encoder.transform([detected_pokemon_name])[0]
        attack_encoded = attack_encoder.transform([attack])[0]

        # Make the prediction
        prediction = model.predict(np.array([[pokemon_encoded, attack_encoded]]))
        predicted_result = result_encoder.inverse_transform(prediction)[0]

        # Define a priority system
        score = 0
        if predicted_result == "super_effective":
            score = 2
        elif predicted_result == "effective":
            score = 1
        elif predicted_result == "not_very_effective":
            score = 0

        # Update the best attack based on the score
        if best_score is None or score > best_score:
            best_score = score
            best_attack = attack

    except Exception as e:
        print(f"Warning: could not predict for attack {attack} -> {str(e)}")

# Display the best attack found
if best_attack:
    print(f"Best attack to use: {best_attack}")
else:
    print("No valid attack found.")